# Further Reading

##  Generic Keywords 
Quantum chemistry packages use gaussian type orbitals (GTO) or  slater type orbitals (STO) to perform the simulation. The packages use the same standards for the basis set and it will be really handy if we can defined a "generic" keyword for basis sets.
Fortunately ``qmflows`` already offers such keyword that can be used among the packages that use the same basis standard,

Internally **QMflows** will create a hierarchical structure representing basis *DZP* for the packages that can handle that basis set.
Other generic keyowrds like: ``functional``, ``inithess``, etc. have been implemented. 

The Following table describes some of the available generic keywords and the Packages where the funcionality is implemented

| Generic  Keyword        | Packages   | Description  |
| :-------------: |:-------------:| :-----:|
|basis | ADF, CP2K, Orca | Set the Basis set |
|cell_angles| CP2K| Specified the angles of the unit cell|
| cell_parameters| CP2K|  Specified the vectors of the unit cell|
| constraint | ADF, Orca| Constrain the distance, angle or dihedral angle for a set of molecules
|freeze| ADF, Gamess, Orca | Freeze a set of atoms indicated by their indexes or symbols|
|functional| ADF, CP2K| Set the DFT functional |
| inithess | ADF, Orca| Provide an initial Hessian matrix|
| optimize| ADF, DFTB, Orca | Perform a molecular optimization|
|selected_atoms| ADF, Gamess, Orca | Optimize the given set of atoms while keeping the rest fixed|
| ts| ADF, DFTB, Orca | Carry out a transition state optimization|

Note: **QMflows** Does not have chemical intuition and if you provide a meaningless keyword,
like a wrong basis set it will not warm you.



## Templates 

As has been shown so far,  **Settings** can be specified in two ways: generic or specific. Generic keywords represent input properties that are present in most simulation packages like a *basis set* while *specific* keywords resemble the input structure of a given package.
 
*Generic* and *Specific* **Settings**  can express both simple and complex simulation inputs,  but it would be nice if we can pre-defined a set of templates for the most common quantum chemistry simulations like: single point calculations, geometry optimizations, transition state optimization, frequency calculations, etc.
*qmflows* already has a pre-defined set of templates  containing some defaults that the user can modify for her/his own purpose. ``Templates`` are stored inside the ``qmflows.templates`` module and are load from *JSON* files. A JSON file is basically a nested dictionary that is translated to a ``Settings`` object by *qmflows*.

Below it is shown the defaults for single point calculation

The question is then, *how I can modify a template with my own changes?*

Suppose you are perfoming a bunch of  constrained *DFT* optimizations using ` ADF ` . You need first to define a basis set  and the constrains.

We use two *generic*  keywords: `freeze` to indicate a constrain and `basis` to provide the basis set. Also, we introduce an specific `ADF` keywords `core = Large`.
Now you merge your **Settings** with the correspoding template to carry out molecular geometry optimizations, using a method called `overlay`.

The ``overlay`` method takes as input a template containing a default set for different packages and also takes the arguments provided by the user, as shown schematically 
<img src="files/merged.png">

This `overlay` method merged the defaults for a given packages (*ADF* in this case) with the input supplied by the user, always given preference to the user input
<img src="files/result_merged.png" width="700">

Below it is shown a combination of templates, generic  and specific keywords to generate the input for a ``CP2K`` job

## Molecule
The next component to carry out a simulation is a molecular geometry.  *qmflows* offers a convinient way to read Molecular geometries using the [Plams](https://www.scm.com/doc/plams/molecule.html) library in several formats like: *xyz* , *pdb*, *mol*, etc.

In [1]:
import os

You can also create the molecule one atom at a time

In [2]:
print("Hello world")

Hello world


**QMflows** Can also handle smiles as shown below

The Molecule class has an extensive functionally to carry out molecular manipulations, for a comprenhesive disccusion about it have a look at the [molecule documentation](https://www.scm.com/doc/plams/molecule.html). Also the module ``qmflows.molkit`` contains an extensive functionality to apply transformation over a molecule using the [RDKit](http://www.rdkit.org/) library.

##  Runinng a quantum mechanics simulation
We now have our components to perform a calculation: **Settings** and **Molecule**. We can now invoke a quantum chemistry package to perform the computation,

the previous code snippet *does not execute the code immediatly*, instead the simulation is started when the user invokes the run function, as shown below
```python
from plams import Molecule
from qmflows import (adf, run, Settings, templates)

# Settings
s = templates.geometry
s.basis = "DZP"
s.specific.adf.basis.core = "Large"
s.freeze = [1, 2, 3]

# molecule 
from plams import Molecule
acetonitrile = Molecule("acetonitrile.xyz")

# Job 
optimized_mol_adf = adf(s, acetonitrile, job_name='acetonitrile_opt')

# run the  job
result = run(optimized_mol_adf.molecule, folder='acetonitrile')
```

you can run the previous script by saving it in a file called `acetonitrile_opt.py` and typing the following command in your console:
```bash
 (qmflows)[user@int1 ~]$ python acetonitrile_opt.py
```

you will then see in your *current work directory* something similar to the following
```bash
 (qmflows)[user@int1 ~]$ ls 
 acetonitrile      acetonitrile_opt.py   cache.json   acetonitrile.xyz  
```

 `acetonitrile` is the folder containing the output from the quantum package call (`ADF` in this case). The `cache.json` file contains all the information required to perform a restart, as we will explore below.
 Inside the `acetonitrile` you can find the input/output files resulting from the simulation
```bash
 (qmflows)[user@int1 ~]$ ls acetonitrile 
 acetonitrile.log  acetonitrile_opt

 (qmflows)[user@int1 ~]$ ls acetonitrile/acetonitrile_opt
  acetonitrile_opt.dill  acetonitrile_opt.out  logfile  t21.N
  acetonitrile_opt.err   acetonitrile_opt.run  t21.C
  acetonitrile_opt.in    acetonitrile_opt.t21  t21.H
```

##   Extracting Properties

In general, properties are extracted using the standard `Object.attribute` notation in python, as shown below.


```python
result = optmized_mol_adf.molecule
```

Some of the available properties are shown in the following table,

| Property        |type           | Description  |
| :-------------: |:-------------:| :-----:|
|dipole        | Double  | Molecular dipole mopment
| energy      | Double | Total energy |
| enthalpy      | Double     |  Enthalpy |
| gradient | Numpy array |  First derivatives of the energy
| hessian | Numpy array| Second derivative of the energy 
|molecule | Molecule | Object representing a physical entity
| runtime | Double      |  Time spent in the simulation |

On the background *QMflows* has a mechanism to read the properties from the output files and make them available inside Python.

## Communicating different packages

We can use the previous optimized geometry for further calculations using for instance another package like *Orca* to run a frequencies calculation,

The whole script is
```python
from qmflows import (adf, orca, run, templates, Settings)
from plams import Molecule
import plams

def main():
    s = templates.geometry
    s.basis = "DZP"
    s.specific.adf.basis.core = "large"

    acetonitrile = Molecule("files/acetonitrile.xyz")
    job = adf(inp, acetonitrile)
    optmized_mol_adf = job.molecule

    s2 = Settings()
    s2.specific.orca.main = "freq"
    s2.specific.orca.basis.basis = 'sto_sz'
    s2.specific.orca.method.functional = 'lda'
    s2.specific.orca.method.method = 'dft'

    job_freq = orca(s2, optmized_mol_adf)
    frequencies = job_freq.frequencies
    
    print(run(frequencies))
```

Once you run the script an input file for the *ADF*  and *Orca* jobs are created. The *ADF*  input looks like

```
Atoms
      1         C      2.419290      0.606560      0.000000 
      2         C      1.671470      1.829570      0.000000 
      3         N      1.065290      2.809960      0.000000 
      4         H      2.000000      0.000000      1.000000 
      5         H      2.000000      0.000000     -1.000000 
      6         H      3.600000      0.800000      0.000000 
End

Basis
  Type DZP
End

Constraints
  Atom 1
  Atom 2
  Atom 3
End

Geometry
  Optim cartesian
End

Integration
  Accint 6.0
End

Scf
  Converge 1e-06
  Iterations 100
End
```

## Running in a supercomputer
Running in **Cartesius** or **Bazis** through the *Slurm* resource manager can be done using and script like

```bash
#!/bin/bash
#SBATCH -t 00:10:00
#SBATCH -N 1
#SBATCH -n 8

module load orca
module load adf/2016.102

source activate qmflows
python optimization_ADF_freq_ORCA.py
```

The Slurm output looks like:

## How the run function works?
###  A little discussion about graphs 

*qmflows* is meant to be used for both workflow generation and execution. When you write a python script representing a workflow you are explicitly declaring  set of computations and their dependencies. For instance the following workflow represent *ADF* and *Orca* computations of the aforementioned example. In this [graph](https://en.wikipedia.org/wiki/Graph_theory) the octagons represent quantum simulation using a package, while the ovals represent both user input or data extracted from a simulation. Finally, the arrows (called edges) represent the dependencies between all these objects.
<img src="files/simple_graph.png">

**QMflows** automatically identify the dependencies between computations and run them in the correct order (if possible in parallel).

### Restarting a simulation

If you are running many computationally expensive calculations in a supercomputer, it can happen that the computations take more time than that allowed by the resource manager in your supercomputer and the workflows gets cancel.  But do not worry, you do not need to re-run all the computations. Fortunately, *QMflows* offers a mechanism to restart the workflow computations.

When running a workflow you will see that *QMflows* creates a set of files called ``cache``. These files contain the information about the workflow and its calculation. **In order to restart a workflow you only need to relaunch it**, that's it!


##  Advanced Examples
###   Conditional Workflows 
```python
from noodles import gather
from qmflows import dftb, adf, orca, run, Settings, templates, molkit, find_first_job

# This examples illustrates the possibility to use different packages interchangeably.
# Analytical frequencies are not available for B3LYP in ADF
# This workflow captures the resulting error and submits the same job to ORCA.

# Define the condition for a successful calculation
def is_successful(result):
    return result.status not in ["failed", "crashed"]

# Generate water molecule
water = molkit.from_smiles('[OH2]', forcefield='mmff')

# Pre-optimize the water molecule
opt_water = dftb(
     templates.geometry, water, job_name="dftb_geometry")

jobs = []

# Generate freq jobs for 3 functionals
for functional in ['pbe', 'b3lyp', 'blyp']:
    s=Settings()
    s.basis = 'DZ'
    s.functional = functional
    # Try to perform the jobs with adf or orca
    # take result from  first successful calculation
    freqjob = find_first_job(
          is_successful, [adf, orca], templates.freq.overlay(s), 
          opt_water.molecule, job_name=functional)
    jobs.append(freqjob)

# Run workflow
results = run(gather(*jobs), n_processes=1)
```

After running the above script you have a table like

    pbe     1533.267   3676.165   3817.097
    b3lyp   1515.799   3670.390   3825.813
    blyp    1529.691   3655.573   3794.110


##  Data flow
```python
Here is a brief introduction of the data flow in the program.

(1) Set up the following information

Create a result dir
Create a video dir 
Create a html dir (stocks)
Create a jobstate file (jobstate.txt)

(2) Parce and submit job

Create a job dir
Parce job input, generate sub file to submit job
Submit job and record job information (jobinfo.txt)
Create a rcgeometry.xyz
Create a jobstate file (jobstate.txt)

(3) Check if new result exist 

Check if job is finished, if so, change “True” to “False” in jobstate.txt, and return jobstate.txt
Check if geometry and converge changed
Create result.txt file to record if any new result generated, if there is any change, return value.

(4) Resub

Initiate jobstate.txt (True or False)in both local and remote directory
After job submittion, jobinfo(jobname) is generated.
Each job check will generate new jobinfo. If converge and geometry is renewed, then will be returned
If jobstate is False, then finish cycle.
```
<img src="files/code_flow.png">